pip install psycopg2-binary
createdb -U postgres clients_db

In [83]:
import psycopg2

def create_db(conn):
    with conn.cursor() as cur:
        cur.execute("""
        CREATE TABLE IF NOT EXISTS clients (
            client_id  SERIAL PRIMARY KEY,
            first_name VARCHAR(60) NOT NULL,
            last_name VARCHAR(60) NOT NULL,
            e_mail  VARCHAR(60) NOT NULL UNIQUE
            CHECK (e_mail ~ '^[a-zA-Z0-9.!#$%&''*+/=?^_`{|}~-]+@[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(?:\.[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?)*$')
            );
        """)
        cur.execute("""
        CREATE TABLE IF NOT EXISTS phones(
            phone_id SERIAL PRIMARY KEY,
            client_id INTEGER REFERENCES clients (client_id) ON DELETE CASCADE,
            phone_number TEXT NOT NULL UNIQUE
            CHECK (phone_number ~ '^\d{10}$')
            );
        """)
        conn.commit()
    return 'Таблицы созданы'

def add_client(conn, first_name, last_name, email, phones=None):
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO clients(first_name, last_name, e_mail) VALUES
            (%s, %s, %s) RETURNING client_id;
            """, (first_name, last_name, email))
        client_id = cur.fetchone()
        if phones == None:
            d = f'Добавлен новый клиент c ID {client_id}, но он не указал телефон'
        else:
            list_phone =[]
            for num_phone in phones.split(','):
                cur.execute("""
                    INSERT INTO phones (client_id, phone_number) VALUES
                    (%s, %s);
                """, (client_id, num_phone.strip()))
                list_phone.append(num_phone.strip())
            conn.commit()
            d = f'Добавлен новый клиент c ID {client_id} и телефонные номера: {"".join(list_phone)}'
    return d

def add_phone(conn, client_id, phone):
    with conn.cursor() as cur:
         cur.execute("""
         INSERT INTO phones (client_id, phone_number) VALUES
        (%s, %s);
        """, (client_id, phone))
    conn.commit()
    return f'Клиенту с ID {client_id} добавлен новый телефонный номер: {phone}'

def change_client(conn, client_id, first_name=None, last_name=None, email=None, phones=None):
     with conn.cursor() as cur:
        cur.execute("""
        UPDATE clients SET first_name = %s, last_name = %s, e_mail = %s
        WHERE client_id=%s
         """, (first_name, last_name, email, client_id))
        cur.execute("""
        SELECT * FROM clients;
        """) 
        d = cur.fetchall()
        return d

In [86]:
with psycopg2.connect(database="clients_db", user="postgres", password="fur2529") as conn:
    #print(create_db(conn))
    #print(add_client(conn, 'Том', 'Круз', 'tomcruser@gmail.com', '9826153947'))
    #print(add_client(conn, 'Дмитрий', 'Нагиев', 'dimadoma@gmail.com', '9826153946'))
    #print(add_client(conn, 'Роберт', 'Де Ниро', 'deniroff@gmail.com'))
    #print(add_client(conn, 'Александр', 'Домогаров', 'adomogarov@gmail.com', '9826153957'))
    #print(add_client(conn, 'Джек', 'Николсон', 'dnick@gmail.com'))
    #print(add_client(conn, 'Павел', 'Деревянко', 'derevo@gmail.com', '9821653957'))
    #print(add_client(conn, 'Анжелина', 'Джоли', 'adjoli@gmail.com'))
    #print(add_client(conn, 'Анастасия', 'Мельникова', 'melana@gmail.com', '9358741237, 9855641248, 9872545186'))
    #print(add_client(conn, 'Павел', 'Прилучный', 'prilucha@gmail.com', '9321653954'))
    #print(add_client(conn, 'Джулия', 'Робертс', 'rjuli@gmail.com', '9312655934'))
    #print(add_phone(conn, 2, '9564237481'))
    print(change_client(conn, 1, 'bezruk@mail.ru'))
    
conn.close()

NotNullViolation: ОШИБКА:  значение NULL в столбце "last_name" отношения "clients" нарушает ограничение NOT NULL
DETAIL:  Ошибочная строка содержит (1, bezruk@mail.ru, null, null).


# обновление данных (U из CRUD)
        cur.execute("""
        UPDATE course SET name=%s WHERE id=%s;
        """, ('Python Advanced', python_id))
        cur.execute("""
        SELECT * FROM course;
        """)
        print(cur.fetchall())  # запрос данных автоматически зафиксирует изменения

def change_client(conn, client_id, first_name=None, last_name=None, email=None, phones=None):
    pass

def delete_phone(conn, client_id, phone):
    pass

def delete_client(conn, client_id):
    pass

def find_client(conn, first_name=None, last_name=None, email=None, phone=None):
    pass


with psycopg2.connect(database="clients_db", user="postgres", password="postgres") as conn:
    pass  # вызывайте функции здесь

conn.close()

In [ ]:
s = '584848848484, 95959595995,    565556565'
for i in s.split(','):
    print(i)
    print(i.strip())